In [2]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                clear all
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                % build path
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                addpath(genpath('../Misc'))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                build_path  niak psom zooniverse_qc  

# Set path
path_root = '/home/yassinebha/Drive/QC_zooniverse/';
#path_root = '/media/yassinebha/database29/Drive/QC_zooniverse/';
path_outline = [path_root 'template_layout/brain_outline_bold/'];
path_layout = [path_root 'template_layout/layout/'];

Adding library niak to the search path.

Adding library psom to the search path.

Adding library zooniverse_qc to the search path.



In [3]:
% invert nask func
in.source = [path_root 'template_layout/test_data/HC0040013/func_HC0040013_mean_stereonl.nii.gz'];
out = [path_root 'template_layout/test_data/HC0040013/HC0040013_test_inv_func.nii.gz'];
opt.only_mask = false;
[in,out,opt] = zoo_brick_invert_contrast(in,out,opt);

In [8]:
% invert mask anat
in.source = [path_root 'template_layout/test_data/HC0040013/anat_HC0040013_nuc_stereonl.nii.gz'];
out = [path_root 'template_layout/test_data/HC0040013/HC0040013_test_inv_anat.nii.gz'];
opt.only_mask = true;
[in,out,opt] = zoo_brick_invert_contrast(in,out,opt);

In [28]:
# anat outline
clear in out opt
in.layout=[path_layout 'layout.nii.gz'];
out = '/home/yassinebha/Drive/QC_zooniverse/template_layout/test_anat_layout.nii.gz';
[in,out,opt]= zoo_brick_outline(in,out);


In [31]:
#func outline
clear in out opt
in.layout=[path_layout 'layout.nii.gz'];
in.mask_func = [path_outline 'func_mask_average_stereonl.mnc.gz'];
opt.modality = 'func';
out = '/home/yassinebha/Drive/QC_zooniverse/template_layout/test_func_layout.nii.gz';

[in,out,opt]= zoo_brick_outline(in,out,opt);

Transforming slices:.............................................................................................................................................................................................Done


In [11]:
#local pipeline report test
clear all
% build path
addpath(genpath('../Misc'))
build_path  niak psom zooniverse_qc  

# Set path
path_root = '/home/yassinebha/Drive/QC_zooniverse/';
#path_root = '/media/yassinebha/database29/Drive/QC_zooniverse/';
path_outline = [path_root 'template_layout/brain_outline_bold/'];
path_layout = [path_root 'template_layout/layout/'];

%HC0040013 
in.ind.anat.HC0040013 = [path_root 'template_layout/test_data/HC0040013/anat_HC0040013_nuc_stereonl.nii.gz'];
in.ind.func.HC0040013  = [path_root 'template_layout/test_data/HC0040013/func_HC0040013_mean_stereonl.nii.gz'];
%HC0040020 
in.ind.anat.HC0040020 = [path_root 'template_layout/test_data/HC0040020/anat_HC0040020_nuc_stereonl.nii.gz'];
in.ind.func.HC0040020  = [path_root 'template_layout/test_data/HC0040020/func_HC0040020_mean_stereonl.nii.gz'];

in.group.mask_func_group = [path_root 'template_layout/test_data/HC0040013/func_mask_group_stereonl.nii.gz'];
in.group.avg_mask_func = [path_root 'template_layout/test_data/group_coregistration/func_mask_average_stereonl_backup.mnc.gz'];
in.template.anat_outline = [path_layout 'layout.nii.gz'];
in.template.func_outline = '';
opt.folder_out = [path_root 'template_layout/test_data/zooqc_adhd200_test2'];
pipeline = zoo_report_fmri_preprocess(in,opt);


Adding library niak to the search path.

Adding library psom to the search path.

Adding library zooniverse_qc to the search path.


Logs will be stored in /home/yassinebha/Drive/QC_zooniverse/template_layout/test_data/zooqc_adhd200_test2/logs/
Generating dependencies ...
   Percentage completed :  0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100- 0.06 sec
Setting up the to-do list ...
   I found 40 job(s) to do.
I could not find any log file. This pipeline has not been started (yet?). Press CTRL-C to cancel.
Deamon started on 11-May-2017 14:51:47
11-May-2017 14:51:47 Starting the pipeline manager...
11-May-2017 14:51:47 Starting the garbage collector...
11-May-2017 14:51:47 Starting worker number 1...
11-May-2017 14:51:47 Starting worker number 2...
11-May-2017 14:51:48 Starting worker number 3...
11-May-2017 14:51:48 Starting worker number 4...
11-May-2017 14:51:48 Starting worker number 5...
11-May-2017 14:51:48 Starting worker number 6...
11-May-2017 14:51:48 Starting 

In [ ]:
#run on guillimin
clear all
path_fmri_preproc ='/home/yassinebha/database/cobre_fmri_preprocess_nii_20160920/';
in = zoo_grab_report_preprocess(path_fmri_preproc);
in.group.avg_mask_func='/home/yassinebha/database/cobre_fmri_preprocess_nii_20160920/quality_control/group_coregistration/func_mask_average_stereonl_backup.mnc.gz';
in.template.anat_outline = '/home/yassinebha/layout.nii.gz';
opt.folder_out =['/home/yassinebha/test_zooqc_pipeline_cobre_' date];
pipeline = zoo_report_fmri_preprocess(in,opt);